# Multi-effect distillation (MED) model development
Alternative using GPy (**Gaussian Process Regresion**) instead of MATLAB's ANNs toolbox

In [1]:
from pathlib import Path
import hjson
import json
import numpy as np
import pandas as pd
import time
from loguru import logger
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import GPy

from phd_visualizations import save_figure
from phd_visualizations.regression import regression_plot
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils.gaussian_process_regresion import GenericModel, GenericModelParameters, train_model, import_train_data

%load_ext autoreload
%autoreload 2

# Paths
data_path: Path = Path("../../data")
results_path = Path("../../results/models_validation")
model_output_path: Path = Path("/workspaces/SolarMED/modeling/src/solarmed_modeling/med/")

# Constants
input_ids: list[str] = ["Ms", "Ts_in", "Mf", "Tc_in", "Tc_out"]
model_input_ids: list[str] = ["Ms", "Ts_in", "Mf", "Tc_in", "Tc_out"]
output_ids: list[str] = ["Md", "Ts_out", "Mc"]
seed: int = 49

assert len(input_ids) == len(model_input_ids), "They need to match in number of elements"


In [2]:
# Load data
with open(data_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)


## Join both experimental campaigns (2020-2022 and 2022-2024)

In [62]:
df_old = pd.read_csv(data_path / 'datasets/med_2009_2012_exp_campaign_operation_points.csv')

df_old


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,2012-04-17 10:47:45+00:00,41.106490,7.787207,1.221267,13.081017,71.829864,67.794467,64.336511,21.368971,44.359224,...,260.556974,229.314313,151.555567,121.170949,95.896905,77.206217,60.501202,45.130574,29.528484,3.30624
841,2012-04-17 10:55:45+00:00,41.104306,7.779161,1.185197,14.131246,71.698753,67.706546,64.280832,21.524272,44.407043,...,259.704036,228.774104,151.542921,121.234136,95.983937,77.289469,60.542123,45.175485,29.530594,3.30624
842,2012-04-17 11:03:45+00:00,41.107775,7.775968,1.201441,15.624742,71.801699,67.759812,64.263024,22.195718,44.246575,...,260.430011,229.121829,151.671249,121.274380,96.003379,77.269748,60.455831,45.009730,29.420515,3.30624
843,2012-04-17 11:11:45+00:00,41.104519,7.766992,1.226294,17.038622,71.894362,67.896687,64.357044,22.605675,44.381693,...,261.433354,229.991126,152.021154,121.638535,96.245695,77.451516,60.582871,45.148548,29.430709,3.30624


In [63]:
# Read and process new experimental campaign so that it has the same structure as the old one

df_new = pd.read_csv(data_path / "datasets/med_2022_2024_exp_campaign_operation_points.csv")
display(df_new)

df_new = df_new[
    ["start_time"]+[f"{var_id}_mean" for var_id in [*input_ids, *output_ids]]
].rename(
    columns=lambda x: x.replace("_mean", "")
).rename(
    columns={"start_time": "time"}
)
df_new


,id,start_time,end_time,duration,PR_value,PR_uncertainty,SEEC_value,SEEC_uncertainty,STEC_value,STEC_uncertainty,...,Lsmp_m_mean,Lsmp_m_uncertainty*,vfd_smp_active_?_mean,vfd_smp_active_?_uncertainty*,Tv_1_C_mean,Tv_1_C_uncertainty*,qsmp_sp_m3h_mean,qsmp_sp_m3h_uncertainty*,Lsmp_sp_m_mean,Lsmp_sp_m_uncertainty*
0,70_27_10_8-20221107,2022-11-07 12:14:40+00:00,2022-11-07 12:21:26+00:00,406.0,4.769537,0.361802,0.649730,0.048830,135.826445,10.435909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74_17_12_8-20230124,2023-01-24 11:12:09+00:00,2023-01-24 11:30:36+00:00,1107.0,6.622377,0.417489,0.386675,0.023875,97.808317,6.196700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74_19_12_8-20230124,2023-01-24 11:50:55+00:00,2023-01-24 12:22:31+00:00,1896.0,7.296894,0.520706,0.355895,0.024505,88.849894,6.364609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60_18_12_8-20230206,2023-02-06 12:06:11+00:00,2023-02-06 12:25:31+00:00,1160.0,6.504494,0.422201,4.891806,0.243135,99.606078,6.456321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60_20_12_8-20230206,2023-02-06 12:35:01+00:00,2023-02-06 13:02:14+00:00,1633.0,9.516285,0.952599,3.430395,0.181498,68.448176,6.801590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,60_28_10_8-20240725,2024-07-25 10:42:48+00:00,2024-07-25 11:10:45+00:00,1677.0,5.351370,1.398079,11.074452,2.960000,129.395388,38.289329,...,1.480217,0.103283,1.0,0.0,55.670606,0.611860,NaN,NaN,NaN,NaN
96,85_30_10_8-20240725,2024-07-25 12:09:58+00:00,2024-07-25 12:30:09+00:00,1211.0,9.599823,1.282492,3.151682,0.469767,68.269263,9.673313,...,1.449933,0.124719,1.0,0.0,75.619825,0.276373,NaN,NaN,NaN,NaN
97,85_28_10_5-20240731,2024-07-31 10:18:06+00:00,2024-07-31 10:39:00+00:00,1254.0,4.041142,1.459891,1.535313,5.928419,176.859585,709.420971,...,1.594641,0.009278,1.0,0.0,76.608326,0.746378,19.389479,1.364619,1.5,0.0
98,85_32_10_9-20240731,2024-07-31 11:16:44+00:00,2024-07-31 12:14:27+00:00,3463.0,9.561755,1.125920,0.982294,0.126825,68.149250,7.998216,...,1.572669,0.016521,1.0,0.0,75.636483,0.251162,16.093962,2.475711,1.5,0.0


,time,Ms,Ts_in,Mf,Tc_out,Tc_in,Md,Ts_out,Mc,Pvc
0,2022-11-07 12:14:40+00:00,10.000684,69.998326,8.002386,26.613328,21.788680,1.347124,65.551291,17.490977,48.935670
1,2023-01-24 11:12:09+00:00,11.998502,74.008899,7.998519,17.425497,10.652321,2.359209,69.325730,18.731883,29.719227
2,2023-01-24 11:50:55+00:00,11.995469,73.966697,8.001771,19.357418,12.172286,2.563332,69.354663,18.060052,32.798986
3,2023-02-06 12:06:11+00:00,11.999940,60.003256,8.001503,17.943657,12.597138,1.637378,56.696498,14.448913,26.787100
4,2023-02-06 12:35:01+00:00,11.996834,60.021941,7.999221,20.001814,13.904699,2.335647,56.773590,12.186765,30.095392
...,...,...,...,...,...,...,...,...,...,...
95,2024-07-25 10:42:48+00:00,10.497901,59.957751,7.997835,27.929277,26.214479,0.850184,57.566087,23.027374,45.603585
96,2024-07-25 12:09:58+00:00,10.497158,84.980477,7.998923,29.981313,25.634669,3.131466,80.073410,20.487897,60.417804
97,2024-07-31 10:18:06+00:00,10.498464,85.004841,5.106164,28.032388,23.524220,1.116329,80.845754,17.552556,55.859308
98,2024-07-31 11:16:44+00:00,10.404054,84.968373,8.795649,32.044899,25.226334,3.177604,79.906265,15.162816,67.154105


In [64]:
data = pd.concat([df_old, df_new], ignore_index=True)
data = data.sort_values(by="time").reset_index(drop=True)

data.to_csv(data_path / "datasets/med_exp_campaigns_operation_points.csv", index=False)
logger.info(f"Saved combined experimental campaigns data to {data_path / 'datasets/med_exp_campaigns_operation_points.csv'}")

data


2025-09-02 10:26:49.371 | INFO     | __main__:<module>:5 - Saved combined experimental campaigns data to ../../data/datasets/med_exp_campaigns_operation_points.csv


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,2024-07-25 10:42:48+00:00,10.497901,7.997835,0.850184,23.027374,59.957751,57.566087,NaN,NaN,45.603585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,2024-07-25 12:09:58+00:00,10.497158,7.998923,3.131466,20.487897,84.980477,80.073410,NaN,NaN,60.417804,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,2024-07-31 10:18:06+00:00,10.498464,5.106164,1.116329,17.552556,85.004841,80.845754,NaN,NaN,55.859308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,2024-07-31 11:16:44+00:00,10.404054,8.795649,3.177604,15.162816,84.968373,79.906265,NaN,NaN,67.154105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Calibrate model

In [3]:
data = pd.read_csv(data_path / "datasets/med_exp_campaigns_operation_points.csv") # "datasets/med_2009_2012_exp_campaign_operation_points.csv")# "datasets/med_exp_campaigns_operation_points.csv")

data["deltaTc"] = data["Tc_out"] - data["Tc_in"]
# Important! Cooling flow measurements prior to october 2021 include extra 4 m³/h from the vacuum system
# data.loc[data["time"] < "2021-10-01", "Mc"] += 4.0

data


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf,deltaTc
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624,11.104826
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624,11.060936
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624,10.889196
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624,10.541512
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624,10.487169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,2024-07-25 10:42:48+00:00,10.497901,7.997835,0.850184,23.027374,59.957751,57.566087,NaN,NaN,45.603585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.714798
941,2024-07-25 12:09:58+00:00,10.497158,7.998923,3.131466,20.487897,84.980477,80.073410,NaN,NaN,60.417804,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.346645
942,2024-07-31 10:18:06+00:00,10.498464,5.106164,1.116329,17.552556,85.004841,80.845754,NaN,NaN,55.859308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.508168
943,2024-07-31 11:16:44+00:00,10.404054,8.795649,3.177604,15.162816,84.968373,79.906265,NaN,NaN,67.154105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.818566


In [9]:
dfs_dict["ref_val"].iloc[[1, 12, 75]]


,Md,Ts_out,Mc,Tc_out
532,2.435117,77.311754,10.614832,35.591412
463,2.570995,65.437889,15.753945,27.869455
534,2.342051,72.906137,11.599375,31.977193


In [4]:
# Filter operation points
"""
Points are considered different and thus valid if:
- ΔTs,in > 0.5 or
- ΔTs,out > 0.5 or
- ΔTvc > 1.5 or
- ΔMs > 0.2*3.6
- ΔMf > 0.5
- ΔTc,out > 0.5
"""

len0 = len(data)
filtered_data = data.copy()
init_len = 0
while init_len != len(filtered_data):
    init_len = len(filtered_data)
    
    # Filter points based on the given conditions
    # filtered_data = data[
    #     (data['Ts_in'].diff().abs() > 0.5) |
    #     (data['Ts_out'].diff().abs() > 0.5) |
    #     (data['Tc_out'].diff().abs() > 1.5) |
    #     (data['Ms'].diff().abs() / 3.6 > 0.2) |
    #     (data['Mf'].diff().abs() > 0.5) |
    #     (data['deltaTc'].diff().abs() > 0.5)
    # ]
    
    # Filter points out of range
    filtered_data = data[
        (data['Ts_in'].between(52, 81)) &
        (data['Tc_out'].between(18, 40)) &
        (data['Tc_in'].between(12, 33.8)) &
        (data['Md'] >= 1.6) &
        (data['Mc'] < 25.) &
        (data['Mf'].between(4.8, 9.5)) &
        (data['deltaTc'].between(2., 25))
    ]

logger.info(f"Filtered {len0 - len(filtered_data)} points, remaining {len(filtered_data)} points.")
display(filtered_data)

data = filtered_data


2025-09-03 16:05:05.514 | INFO     | __main__:<module>:39 - Filtered 396 points, remaining 549 points.


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf,deltaTc
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624,11.104826
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624,11.060936
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624,10.889196
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624,10.541512
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624,10.487169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,2024-01-23 11:33:39+00:00,12.005303,5.000163,1.849893,12.172505,60.004398,57.270397,NaN,NaN,29.136047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.619666
934,2024-02-15 13:12:58+00:00,12.007063,4.998687,2.733578,10.025680,79.987560,76.255694,NaN,NaN,43.886432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.163333
935,2024-02-15 13:27:24+00:00,11.998619,5.003968,2.720984,10.006947,79.982145,76.325461,NaN,NaN,46.120058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.910661
936,2024-02-15 13:51:13+00:00,11.995167,4.998590,2.661354,10.001759,79.936675,76.337803,NaN,NaN,47.889175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.840472


In [5]:
params_arrays, _, dfs_dict = train_model(
    data=data, 
    input_ids=input_ids, 
    output_ids=output_ids, 
    model_input_ids=model_input_ids, 
    model_output_path=model_output_path / "med_gpr_data.npy"
)


Ms: 10.495227543376014 - 44.34859111520107
Ts_in: 52.00969184692181 - 80.97849778943828
Mf: 4.975868611910029 - 8.265407814949919
Tc_in: 12.140060565723797 - 33.63826156405986
Tc_out: 19.357417859535204 - 39.886232113144736
Md: 1.6083078535773712 - 3.075405940099833
Ts_out: 50.05487470881863 - 77.31175386314521
Mc: 8.02892778702163 - 23.125029940119767
Ms: 10.495227543376014 - 44.34859111520107
Ts_in: 52.00969184692181 - 80.97849778943828
Mf: 4.975868611910029 - 8.265407814949919
Tc_in: 12.140060565723797 - 33.63826156405986
Tc_out: 19.357417859535204 - 39.886232113144736
X.shape=(549, 5), Y.shape=(549, 3)
X_train.shape=(439, 5), Y_train.shape=(439, 3)
X_val.shape=(110, 5), Y_val.shape=(110, 3)


Model for output 'Md':

Name : GP regression
Objective : -336.1291859298012
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |                 value  |  constraints  |  priors
  rbf.variance             |    1.9867205543419775  |      +ve      |        
  rbf.lengthscale          |     6.693755214726311  |      +ve      |        
  Gaussian_noise.variance  |  0.004159348871379535  |      +ve      |        
model.param_array=array([1.98672055e+00, 6.69375521e+00, 4.15934887e-03])


Model for output 'Ts_out':

Name : GP regression
Objective : -180.84163453955279
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |                 value  |  constraints  |  priors
  rbf.variance             |     999.8787080345198  |      +ve      |        
  rbf.lengthscale          |     27.55730316759485  |      +ve      |        
  Gaussian_noise.variance  |  0.011559108440227887  |      +ve      |        
model.param_array=array([9.99878708e+02, 2.75573032e+01, 1.15591084e-02])


Model for output 'Mc':

Name : GP regression
Objective : 661.4536160356835
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |                value  |  constraints  |  priors
  rbf.variance             |   116.61624543437144  |      +ve      |        
  rbf.lengthscale          |    5.938738928616871  |      +ve      |        
  Gaussian_noise.variance  |  0.39178961211905033  |      +ve      |        
model.param_array=array([116.61624543,   5.93873893,   0.39178961])
params_arrays=[(1.9867205543419775, 6.693755214726311, 0.004159348871379535), (999.8787080345198, 27.55730316759485, 0.011559108440227887), (116.61624543437144, 5.938738928616871, 0.39178961211905033)]


/workspaces/solarmed/miniconda3/envs/conda-env/lib/python3.11/site-packages/paramz/parameterized.py:61: RuntimeWarning:

Don't forget to initialize by self.initialize_parameter()!



In [6]:
params_arrays_aux, _, dfs_dict_aux = train_model(
    data=data, 
    input_ids=["Ms", "Ts_in", "Mf", "Tc_in", "Mc"], 
    model_input_ids=["Ms", "Ts_in", "Mf", "Tc_in", "Mc"], 
    output_ids=["Tc_out"], 
    model_output_path=model_output_path / "med_gpr_data_aux.npy", 
    train_indices=dfs_dict["ref_train"].index.values,
    val_indices=dfs_dict["ref_val"].index.values,
)


Ms: 10.495227543376014 - 44.34859111520107
Ts_in: 52.00969184692181 - 80.97849778943828
Mf: 4.975868611910029 - 8.265407814949919
Tc_in: 12.140060565723797 - 33.63826156405986
Mc: 8.02892778702163 - 23.125029940119767
Tc_out: 19.357417859535204 - 39.886232113144736
Ms: 10.495227543376014 - 44.34859111520107
Ts_in: 52.00969184692181 - 80.97849778943828
Mf: 4.975868611910029 - 8.265407814949919
Tc_in: 12.140060565723797 - 33.63826156405986
Mc: 8.02892778702163 - 23.125029940119767
X.shape=(549, 5), Y.shape=(549, 1)
X_train.shape=(439, 5), Y_train.shape=(439, 1)
X_val.shape=(110, 5), Y_val.shape=(110, 1)


Model for output 'Tc_out':

Name : GP regression
Objective : 314.61072793031707
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |                value  |  constraints  |  priors
  rbf.variance             |    385.2097576747307  |      +ve      |        
  rbf.lengthscale          |    21.78824050066122  |      +ve      |        
  Gaussian_noise.variance  |  0.14051420169703088  |      +ve      |        
model.param_array=array([3.85209758e+02, 2.17882405e+01, 1.40514202e-01])
params_arrays=[(385.2097576747307, 21.78824050066122, 0.14051420169703088)]


/workspaces/solarmed/miniconda3/envs/conda-env/lib/python3.11/site-packages/paramz/parameterized.py:61: RuntimeWarning:

Don't forget to initialize by self.initialize_parameter()!



In [7]:
# Join the two datasets
for key in dfs_dict.keys():
    dfs_dict[key] = pd.concat([dfs_dict[key], dfs_dict_aux[key]], axis=1)
dfs_dict["out_train"]


,Md,Ts_out,Mc,Tc_out
195,2.726423,68.121120,16.144078,31.915855
79,1.741784,52.873793,9.531200,29.378072
479,2.218170,59.507231,16.913691,22.744106
109,2.710922,69.845955,14.948468,30.977755
473,2.724170,70.898899,10.311276,31.695818
...,...,...,...,...
391,2.223188,61.757632,17.065413,30.254855
489,2.298633,61.059385,21.116042,22.433552
138,1.892855,55.021079,8.355371,29.853081
62,1.772430,53.680643,8.437982,29.999505


In [ ]:
width=1200
height=400

fig = regression_plot(
    df_ref=dfs_dict["ref_val"].drop(dfs_dict["ref_val"].index[[1, 12, 75]]),
    df_mod=dfs_dict["out_val"].drop(dfs_dict["out_val"].index[[1, 12, 75]]),
    df_ref_bg = dfs_dict["ref_train"],
    df_mod_bg = dfs_dict["out_train"],
    show_error_metrics=["r2", "mae"],
    var_ids=output_ids + ["Tc_out"],
    var_labels=["q<sub>med,d</sub>", "T<sub>med,s,out</sub>", "q<sub>med,c</sub>", "T<sub>med,c,out</sub>"],
    units=["m³/h", "°C", "m³/h", "°C"],
    instruments=["vortex_flow_meter", "pt100", "vortex_flow_meter", "pt100"],
    bg_label="Training set",
    legend_pos="side",
    title_y=0.99,
    figure_layout="horizontal",
    width=width, height=height,
    title_text=""
)

fig.show()

save_figure(
    fig,
    figure_path=results_path,
    figure_name="med_model_regression_plot.png",
    formats=["png", "svg", "html"]
)


2025-09-03 16:34:09.873 | INFO     | phd_visualizations:save_figure:41 - Figure saved in /workspaces/SolarMED/modeling/src/solarmed_modeling/med/med_model_regression_plot.png
2025-09-03 16:34:12.136 | INFO     | phd_visualizations:save_figure:41 - Figure saved in /workspaces/SolarMED/modeling/src/solarmed_modeling/med/med_model_regression_plot.svg
2025-09-03 16:34:12.170 | INFO     | phd_visualizations:save_figure:41 - Figure saved in /workspaces/SolarMED/modeling/src/solarmed_modeling/med/med_model_regression_plot.html


In [ ]:
from dataclasses import dataclass, field
from typing import Optional
from iapws import IAPWS97 as w_props
    
@dataclass
class FixedModelParameters:
    """
    MED fixed model parameters.
    Default values could be replaced by values from a configuration file. 
    """
    param_array: list[tuple[float, float, float]]
    param_array_aux: list[tuple[float, float, float]]
    
    Tmed_s_min: float = 55  # Minimum operating heat source temperature [ºC]
    Tmed_s_max: float = 81  # Maximum operating heat source temperature [ºC]
    qmed_c_min: float = 3   # Minimum cooling flow rate [m³/h]
    qmed_c_max: float = 22  # Maximum cooling flow rate [m³/h]
    qmed_s_min: float = 30  # Minimum heat source flow rate [m³/h]
    qmed_s_max: float = 48  # Maximum heat source flow rate [m³/h]
    qmed_f_min: float = 5   # Minimum feed flow rate [m³/h]
    qmed_f_max: float = 9   # Maximum feed flow rate [m³/h]
    deltaTc_range: tuple[float, float] = (1.5, 25)  # Condenser inlet-outlet temperature difference range [°C]

    # param_array: list[tuple[float, float, float]] = field(default_factory=lambda: # qd, Ts,out, qc = f(qs, Ts,in, qf, Tc,in, Tc,out)
    #     [(2.7370501116935837, 17.60540080548287, 0.009364821318621382), (1275.2229954536995, 33.4485753565255, 0.01320819225299155), (116.30078564132958, 6.770856543200832, 0.4723189021752448), (7871.537995629091, 31.566575608184507, 2.729211750775973)]
    # )
    # param_array_aux: list[tuple[float, float, float]] = field(default_factory=lambda: # Tc,out = f(qs, Ts,in, qf, Tc,in, qc)
    #     [(2.7370501116935837, 17.60540080548287, 0.009364821318621382)]
    # )

@dataclass
class MedModel:
    """
    MED GPR model wrapper.
    Outputs: qd, Ts,out, qc
    Inputs: Ms, Ts,in, Mf, Tc,in, Tc,out
    """
    
    models: Optional[list[GPy.models.GPRegression]] = None
    models_aux: Optional[list[GPy.models.GPRegression]] = None
    train_data_path: Optional[Path] = None
    fmp: FixedModelParameters = field(default_factory=lambda: FixedModelParameters())
    n_outputs: int = 3
    
    def __post_init__(self):
        
        if self.train_data_path is None:
            self.train_data_path = Path(__file__).resolve().parent 
            
        X, Y = import_train_data(self.train_data_path / 'med_gpr_data.npy', ydim=self.n_outputs)
        Xaux, Yaux = import_train_data(self.train_data_path / 'med_gpr_data_aux.npy', ydim=1)            
        
        self.models = self._create_models(self.fmp.param_array, X, Y)
        self.models_aux = self._create_models(self.fmp.param_array_aux, Xaux, Yaux)
    
    def _create_models(self, param_array: list[tuple[float, ...]], X: np.ndarray[float], Y: np.ndarray[float]) -> list[GPy.models.GPRegression]:
        """
        Create GPR models for each output using the provided training data and fixed model parameters.
        """
        models = []
        for i, param in enumerate(param_array):
            model = GPy.models.GPRegression(X, Y[:, [i]], initialize=False)
            model.update_model(False) # do not call the underlying expensive algebra on load
            model.initialize_parameter() # Initialize the parameters (connect the parameters up)
            model[:] = param # Load the parameters
            model.update_model(True) # Call the algebra only once
            models.append(model)
            
        return models
    
    def predict(self, X: np.ndarray, return_variances: bool = False, aux: bool = False) -> np.ndarray[float] | tuple[np.ndarray[float], np.ndarray[float]]:
        
        models = self.models if not aux else self.models_aux
        Y_preds, Y_pred_vars = zip(*(model.predict(X) for model in models))
        Y_preds = np.hstack(Y_preds)  # Combine predictions
        
        if return_variances:
            return Y_preds, np.hstack(Y_pred_vars)  # Combine variances
        return Y_preds
    
    def __call__(self, qs_m3h: float, Ts_in_C: float, qf_m3h: float, Tc_in_C: float, Tc_out_C: float, debug: bool = False) -> tuple[float, float, float, float]:
        """
        Evaluate the model for given inputs.
        
        Returns:
            Tuple containing (qd_m3h, Ts_out_C, qc_m3h, Pvc_mbar)
        """
                
        if (
            qf_m3h > 0 and 
            qs_m3h > 0 and 
            Ts_in_C > 0 # and
            # (Tc_in_C+self.fmp.deltaTc_range[0] < Tc_out_C < Tc_in_C+self.fmp.deltaTc_range[1]) 
        ):
            
            qd_m3h, Ts_out_C, qc_m3h = self.predict(
                np.array([[qs_m3h, Ts_in_C, qf_m3h, Tc_in_C, Tc_out_C]])    
            ).flatten()
            
            qc_m3h -= 5.

            # qc_m3h = qd_m3h * (4.18 * (Tc_in_C - Tc_out_C)) / (w_props(P=Pvc_mbar/1000, x=0).h -w_props(P=Pvc_mbar/1000, x=1).h)
            # if debug:
            #     return qd_m3h, Ts_out_C, qc_m3h, Pvc_mbar, qc_m3h_gpr
            
            if self.fmp.qmed_c_min <= qc_m3h <= self.fmp.qmed_c_max:
                return qd_m3h, Ts_out_C, qc_m3h, Tc_out_C # Model solved
            
            if qc_m3h > self.fmp.qmed_c_max:
                Tc_out_C = self.predict(
                    np.array([[qs_m3h, Ts_in_C, qf_m3h, Tc_in_C, self.fmp.qmed_c_max]]
                ), aux=True).flatten()[0]
            elif qc_m3h < self.fmp.qmed_c_min:
                Tc_out_C = self.predict(
                    np.array([[qs_m3h, Ts_in_C, qf_m3h, Tc_in_C, self.fmp.qmed_c_min]]
                ), aux=True).flatten()[0]
            
            qd_m3h, Ts_out_C, qc_m3h = self.predict(
                np.array([[qs_m3h, Ts_in_C, qf_m3h, Tc_in_C, Tc_out_C]])    
            ).flatten()
            
            return qd_m3h, Ts_out_C, qc_m3h, Tc_out_C
            
        else:
            return [np.nan]*(self.n_outputs+1)


# Both options should work
fmp = FixedModelParameters(param_array=params_arrays, param_array_aux=params_arrays_aux)
# X,Y = import_train_data(model_output_path, ydim=len(output_ids))
# med_model = MedModel(X=X, Y=Y, fsm=fsm) # Provide X and Y
med_model = MedModel(train_data_path=model_output_path, fmp=fmp) # Provide path to load X and Y

# # Evaluate validation dataset
# Y_preds, Y_pred_vars = med_model.predict(X_val, return_variances=True)

# print(f"{X_val.shape=}")
# print(f"{Y_preds[:, 0].shape=}")
# print(f"{Y_preds.shape=}")
# print(f"{Y_pred_vars.shape=}")


## Evaluate model


In [3]:
# Evaluate some experimental timeseries
from solarmed_modeling.data_validation import (within_range_or_nan_or_max, 
                                               within_range_or_zero_or_max,
                                               within_range_or_nan)
from solarmed_modeling.utils import data_preprocessing, data_conditioning

# Comment this part to use the model created above
from solarmed_modeling.med import MedModel, FixedModelParameters
fmp = FixedModelParameters()
med_model = MedModel(fmp=fmp)

logger.disable("phd_visualizations.utils.units")

date_str: str = "20230331" # "20230418" # "20230414" # "20230331" # "20230630" # "20230511"  # '20230630'
sample_rate: int = 60

# Load data and preprocess data
df = data_preprocessing([data_path / f"datasets/{fn}" for fn in [f"{date_str}_MED.csv"]], # f"{date_str}_solarMED.csv"
                        vars_config, sample_rate_key=f"{sample_rate}s")
# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate, vars_config=vars_config,)

df


2025-09-08 09:26:19.776 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id RE-SF-001 not found in dataframe columns.
2025-09-08 09:26:19.777 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id TT-SF-009 not found in dataframe columns.
2025-09-08 09:26:19.777 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id TT-SF-010 not found in dataframe columns.
2025-09-08 09:26:19.778 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id FT-SF-002 not found in dataframe columns.
2025-09-08 09:26:19.778 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id TT-SF-008 not found in dataframe columns.
2025-09-08 09:26:19.779 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id TT-SF-008 not found in dataframe columns.
2025-09-08 09:26:19.780 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id

,Jmed_d,vfd_prod_freq_EEPROM,vfd_brine_input_freq,vfd_sw_input_freq,vfd_prod_active,qmed_c,vfd_brine_freq_EEPROM,vfd_prod_current_peak,vfd_prod_input_freq,vfd_brine_current_peak,...,TT-DES-010,TT-DES-005,FT-AQU-102,PT-DES-007,PT-DES-011,qmed_f,T3wv_dis_in,q3wv_dis,Jmed,Jtotal
time,,,,,,,,,,,,,,,,,,,,,
2023-03-31 10:27:00+00:00,0.0,0.0,0.000000,29.259259,0.000000,11.711794,0.0,6.97,0.0,5.360000,...,23.787938,27.494858,0.008681,35.590172,27.263380,0.000000,51.664577,0.000000,6.973478,6.973478
2023-03-31 10:28:00+00:00,0.0,0.0,35.618760,30.000000,1.000000,10.336187,0.0,6.97,0.0,5.204407,...,23.768294,27.505837,0.008681,35.590172,27.202508,0.000000,46.072785,0.000000,8.134073,8.134073
2023-03-31 10:29:00+00:00,0.0,0.0,30.167110,30.000000,1.000000,10.416375,0.0,6.97,0.0,5.190000,...,23.740598,27.654081,0.008681,35.590172,27.217885,0.000000,42.621889,0.000000,7.594104,7.594104
2023-03-31 10:30:00+00:00,0.0,0.0,29.266991,30.000000,1.000000,10.378185,0.0,6.97,0.0,5.190000,...,23.204813,27.508680,0.008662,34.939271,26.563948,7.689519,41.012310,0.000000,7.416302,7.416302
2023-03-31 10:31:00+00:00,0.0,0.0,29.170239,30.000000,1.000000,10.362726,0.0,6.97,0.0,5.190000,...,21.367408,25.638536,0.009113,34.310300,25.616468,7.062256,38.213878,35.959481,7.366027,7.366027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 14:13:00+00:00,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,6.92,0.0,5.170000,...,35.462962,38.352141,0.009766,266.116914,256.394683,0.000000,44.920609,0.000000,0.017183,0.017183
2023-03-31 14:14:00+00:00,0.0,0.0,0.000000,0.000000,0.033333,0.000000,0.0,6.92,0.0,5.083833,...,35.462479,38.427614,0.009757,280.671307,272.022582,0.000000,44.855505,0.000000,0.017733,0.017733
2023-03-31 14:15:00+00:00,0.0,0.0,16.145457,0.000000,0.000000,0.000000,0.0,6.92,0.0,5.211167,...,35.464891,38.467638,0.009766,295.471631,287.063093,0.000000,44.645785,0.000000,0.669767,0.669767


In [4]:
idx_start = 10
idx_end = len(df)
df_mod = pd.DataFrame()

# Run model
for idx in range(idx_start,idx_end):
    ds = df.iloc[idx]
        
    start_time = time.time()

    Tmed_c_out = ds["Tmed_c_out"]
    Tmed_c_in = ds["Tmed_c_in"]
    mmed_f = ds["qmed_f"]
    mmed_f = within_range_or_nan_or_max(mmed_f, range=(fmp.qmed_f_min, fmp.qmed_f_max), var_name="qmed_f")
    mmed_s = within_range_or_zero_or_max(ds["qmed_s"], range=(fmp.qmed_s_min, fmp.qmed_s_max), var_name="qmed_s")
    Tmed_s_in = within_range_or_nan_or_max(ds["Tmed_s_in"], range=(fmp.Tmed_s_min, fmp.Tmed_s_max), var_name="Tmed_s_in")
    # deltaTc = within_range_or_nan(Tmed_c_out - Tmed_c_in, range=(2., 25), var_name="deltaTc")
    
    qmed_d, Tmed_s_out, qmed_c, Tmed_c_out = med_model(
        qs_m3h=mmed_s,
        Ts_in_C=Tmed_s_in,
        qf_m3h=mmed_f,
        Tc_in_C=Tmed_c_in,
        Tc_out_C=Tmed_c_out,
        debug=True
    )
    logger.info(f"Iteration {idx} / {len(df)} | Elapsed time: {time.time()-start_time:.2f} seconds | qmed_d: {qmed_d:.2f} m³/h")
    
    result = pd.DataFrame(
        {
            "qmed_f": mmed_f,
            "qmed_d": qmed_d,
            "qmed_c": qmed_c,
            "qmed_s": mmed_s,
            "Tmed_s_in": Tmed_s_in,
            "Tmed_s_out": Tmed_s_out, 
            "Tmed_c_in": ds["Tmed_c_in"],
            "Tmed_c_out": Tmed_c_out,
        },
        index=[0]
    )
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)

# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

with open(data_path / 'plt_config_med.json') as f:
    plt_config = json.load(f)
    
fig = experimental_results_plot(
    plt_config, 
    df, 
    df_comp=df_mod, 
    vars_config=vars_config, 
    resample=False,
    comp_trace_labels=[f"[Ts={sample_rate}s]"]
)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'med_validation_{df.index[0].strftime("%Y%m%d")}')
)


2025-09-08 09:26:25.346 | INFO     | __main__:<module>:27 - Iteration 10 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.28 m³/h
2025-09-08 09:26:25.352 | INFO     | __main__:<module>:27 - Iteration 11 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.29 m³/h
2025-09-08 09:26:25.356 | INFO     | __main__:<module>:27 - Iteration 12 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.30 m³/h
2025-09-08 09:26:25.360 | INFO     | __main__:<module>:27 - Iteration 13 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.31 m³/h
2025-09-08 09:26:25.364 | INFO     | __main__:<module>:27 - Iteration 14 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.33 m³/h
2025-09-08 09:26:25.367 | INFO     | __main__:<module>:27 - Iteration 15 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.34 m³/h
2025-09-08 09:26:25.370 | INFO     | __main__:<module>:27 - Iteration 16 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.39 m³/h
2025-09-08 09:26:25.372 | INFO     | __main__:<module>:27 - Iteration 17 / 231 | Elapsed time: 0.00 seco

2025-09-08 09:26:25.438 | INFO     | __main__:<module>:27 - Iteration 44 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.55 m³/h
2025-09-08 09:26:25.440 | INFO     | __main__:<module>:27 - Iteration 45 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.57 m³/h
2025-09-08 09:26:25.442 | INFO     | __main__:<module>:27 - Iteration 46 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.59 m³/h
2025-09-08 09:26:25.444 | INFO     | __main__:<module>:27 - Iteration 47 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.61 m³/h
2025-09-08 09:26:25.447 | INFO     | __main__:<module>:27 - Iteration 48 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.63 m³/h
2025-09-08 09:26:25.451 | INFO     | __main__:<module>:27 - Iteration 49 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.68 m³/h
2025-09-08 09:26:25.454 | INFO     | __main__:<module>:27 - Iteration 50 / 231 | Elapsed time: 0.00 seconds | qmed_d: 2.72 m³/h
2025-09-08 09:26:25.461 | INFO     | __main__:<module>:27 - Iteration 51 / 231 | Elapsed time: 0.00 seco

### Generate stacked figure with calibration and validation tests


In [42]:
import pandas as pd
from pathlib import Path
from phd_visualizations.utils import stack_images_horizontally, update_plot_config

data_path = Path("../../data")
results_path = Path("../../results/models_validation")


In [47]:
# Generate partial images to later stack them horizontally
# 1. Run the simulation above for a test.
# 2. Then run this cell to generate the image
# 3. Repeat with a different test (Remeber to change the idx variable below!)

idx = 0  # Change this to 0, 1, 2, ... for different images

plt_config = json.load(open(data_path / 'plt_config_med.json'))

if idx == 0:
    show_titles = True
    showlegends=False
    show_left_axis_titles = True
    show_right_axis_titles = True
    
    width = 600
    title_text=f"<b>MED</b> model validation<br>T<sub>s</sub>={sample_rate}s</span>"
    # pos_keys = ["top", "med", "bottom"]
    # for pos_key, tank_key in product(pos_keys, ["hot", "cold"]):
    #     plot_id = f"temperatures_{tank_key}_{pos_key}"
    #     key_idx = pos_keys.index(pos_key)
    #     plot_config["plots"][plot_id]["title"] = f"H={getattr(model_params, f'UA_{tank_key[0]}')[key_idx]:.2e} W/K, V={getattr(model_params, f'V_{tank_key[0]}')[key_idx]:.2f} m³"
    #     plot_config["plots"][plot_id]["ylabels_left"] = [f"{tank_key.capitalize()} tank<br>{pos_key.capitalize()}<br>°C"]
else:
    show_titles = False
    showlegends=True
    show_left_axis_titles = False
    show_right_axis_titles = False
    width = 700
    title_text=""

fig = experimental_results_plot(
    update_plot_config(plt_config, show_titles, show_left_axis_titles, show_right_axis_titles, showlegends, width), 
    df, 
    df_comp=df_mod, 
    vars_config=vars_config, 
    resample=False,
    comp_trace_labels=[f"[Ts={sample_rate}s]"],
    title_text=title_text
)

fig.show()

save_figure(
    fig,
    figure_path=results_path,
    figure_name=f'fig_med_{idx}',
    formats=["png"]
)


2025-09-03 09:04:50.741 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/fig_med_0.png


In [48]:
stack_images_horizontally(
    image_paths = [results_path / "fig_med_0.png", results_path / "fig_med_1.png" ],
    output_path=results_path / "med_validation.png",
)


### Evaluate and generate visualizations for model with different sample rates and for many dates

(This cell is self-contained and can be run without running previous cells.)

In [1]:
from pathlib import Path
from solarmed_modeling.med import FixedModelParameters
from solarmed_modeling.med.benchmark import benchmark_model
from loguru import logger

logger.enable("solarmed_modeling")
logger.disable("solarmed_modeling.data_validation")

data_path: Path = Path("../../data")
results_path: Path = Path("../../results/models_validation")

stats = benchmark_model(
    fixed_model_params=FixedModelParameters(),
    data_path=data_path,
    output_path=results_path,
    sample_rates=[5, 400],
    save_results=True,
    skip_sample_rates_viz=[5],
)


2025-09-02 18:13:25.244 | INFO     | solarmed_modeling.benchmark:benchmark_model:93 - Processing test 20230414 (1/5)
2025-09-02 18:13:25.245 | INFO     | solarmed_modeling.utils:data_preprocessing:113 - Reading data from 20230414_MED.csv


Processing test 20230414 (1/5)


2025-09-02 18:13:25.566 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-09-02 18:13:25.568 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 74
2025-09-02 18:13:25.604 | WARNING  | solarmed_modeling.utils:data_conditioning:204 - Skipping estimation of qhx_s, missing required columns: ['Thx_p_in', 'Thx_s_in', 'Thx_p_out', 'Thx_s_out', 'qhx_p']
2025-09-02 18:13:25.605 | WARNING  | solarmed_modeling.utils:data_conditioning:253 - Skipping estimation of qts_dis, missing required columns: ['T3wv_dis_out', 'T3wv_src']
2025-09-02 18:13:25.607 | INFO     | solarmed_modeling.utils:calculate_aux_variables:515 - MED electricity power consumption calculated successfully.
2025-09-02 18:13:25.608 | WARNING  | solarmed_modeling.utils:calculate_aux_variables:521 - Solar field and thermal storage power consumption not found, total electricity power consumption will be calculated without it.
2025-09-02 18:13:25.6

Processing test 20230418 (2/5)


2025-09-02 18:13:37.960 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-09-02 18:13:37.962 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 59
2025-09-02 18:13:38.002 | WARNING  | solarmed_modeling.utils:data_conditioning:204 - Skipping estimation of qhx_s, missing required columns: ['Thx_p_in', 'Thx_s_in', 'Thx_p_out', 'Thx_s_out', 'qhx_p']
2025-09-02 18:13:38.003 | WARNING  | solarmed_modeling.utils:data_conditioning:253 - Skipping estimation of qts_dis, missing required columns: ['T3wv_dis_out', 'T3wv_src']
2025-09-02 18:13:38.004 | INFO     | solarmed_modeling.utils:calculate_aux_variables:515 - MED electricity power consumption calculated successfully.
2025-09-02 18:13:38.005 | WARNING  | solarmed_modeling.utils:calculate_aux_variables:521 - Solar field and thermal storage power consumption not found, total electricity power consumption will be calculated without it.
2025-09-02 18:13:38.0

Processing test 20231030 (3/5)
Processing test 20230331 (4/5)


2025-09-02 18:13:50.312 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-09-02 18:13:50.314 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 59
2025-09-02 18:13:50.358 | WARNING  | solarmed_modeling.utils:data_conditioning:204 - Skipping estimation of qhx_s, missing required columns: ['Thx_p_in', 'Thx_s_in', 'Thx_p_out', 'Thx_s_out', 'qhx_p']
2025-09-02 18:13:50.359 | WARNING  | solarmed_modeling.utils:data_conditioning:253 - Skipping estimation of qts_dis, missing required columns: ['T3wv_dis_out', 'T3wv_src']
2025-09-02 18:13:50.360 | INFO     | solarmed_modeling.utils:calculate_aux_variables:515 - MED electricity power consumption calculated successfully.
2025-09-02 18:13:50.361 | WARNING  | solarmed_modeling.utils:calculate_aux_variables:521 - Solar field and thermal storage power consumption not found, total electricity power consumption will be calculated without it.
2025-09-02 18:13:50.3

Processing test 20231106 (5/5)


2025-09-02 18:14:01.273 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-09-02 18:14:01.275 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-09-02 18:14:01.275 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-09-02 18:14:01.328 | WARNING  | solarmed_modeling.utils:data_conditioning:204 - Skipping estimation of qhx_s, missing required columns: ['Thx_p_in', 'Thx_s_in', 'Thx_p_out', 'Thx_s_out', 'qhx_p']
2025-09-02 18:14:01.329 | WARNING  | solarmed_modeling.utils:data_conditioning:253 - Skipping estimation of qts_dis, missing required columns: ['T3wv_dis_out', 'T3wv_src']
2025-09-02 18:14:01.330 | INFO     | solarmed_modeling.utils:calculate_aux_variables:515 - MED electricity power consumption calculated successfully.
2025-09-02 18:14:01.331 | WARNING  | solarmed_modeling.utils:calculate_aux_variables:521 - Solar field and thermal 